This is a port of the [tensorflow compression tutorial](https://www.tensorflow.org/tutorials/generative/data_compression) using:
* [Flax](https://flax.readthedocs.io/en/latest/)
* [PIX](https://github.com/deepmind/dm_pix)
* [Codex](https://github.com/google/codex)
* [Huggingface datasets](https://huggingface.co/docs/datasets/use_with_jax)

In [1]:
import flax.linen as nn
import jax
import jax.numpy as jnp

In [2]:
from flax import linen as nn
from flax.training import train_state
from flax.training.common_utils import get_metrics, onehot
from flax.metrics import tensorboard
import optax
import jax
from jax import numpy as jnp
from jax import random
import codex.ems

2023-07-18 05:22:47.758356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datasets import load_dataset
import dm_pix as pix
import PIL.Image as Image
import numpy as onp
from IPython.display import display

In [4]:
from flax.training import train_state
from flax.training.common_utils import shard, get_metrics
import time

In [23]:
class AnalysisTransform(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Conv(features=20, kernel_size=(5, 5), strides=(2, 2))(x)
        x = nn.leaky_relu(x)
        x = nn.Conv(features=50, kernel_size=(5, 5), strides=(2, 2))(x)
        x = nn.leaky_relu(x)
        x = x.reshape((x.shape[0], -1))
        x = nn.Dense(features=500)(x)
        x = nn.leaky_relu(x)
        x = nn.Dense(features=2450)(x)
        return x

class SynthesisTransform(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=500)(x)
        x = nn.leaky_relu(x)
        x = nn.Dense(features=2450)(x)
        x = nn.leaky_relu(x)
        x = x.reshape((x.shape[0], 7, 7, 50))
        x = nn.ConvTranspose(features=20, kernel_size=(5, 5), strides=(2, 2))(x)
        x = nn.leaky_relu(x)
        x = nn.ConvTranspose(features=1, kernel_size=(5, 5), strides=(2, 2))(x)
        x = nn.leaky_relu(x)
        return (x)

In [6]:
ds = load_dataset("mnist")
ds = ds.with_format("jax")
test_image = ds['train'][0]['image']
display(Image.fromarray(onp.array(test_image)))
batch = jnp.expand_dims(test_image, 0)
batch = jnp.expand_dims(batch, 3)

Found cached dataset mnist (/home/server/.cache/huggingface/datasets/mnist/mnist/1.0.0/9d494b7f466d6931c64fb39d58bb1249a4d85c9eb9865d9bc20960b999e2a332)


  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
h = AnalysisTransform()
h_vars = h.init(jax.random.PRNGKey(0), batch)
z = h.apply(h_vars, batch)

In [25]:
g = SynthesisTransform()
g_vars = g.init(jax.random.PRNGKey(0), z)
y = h.apply(g_vars,z)

In [67]:
h(test_image)

ValueError: parent must be None, Module or Scope

In [4]:
class AnalysisTransform(nn.Module):
    latent_dims: int

    def setup(self):
        self.conv_1 = nn.Conv(features=20, kernel_size=(5, 5), strides=(2, 2))
        self.conv_2 = nn.Conv(features=50, kernel_size=(5, 5), strides=(2, 2))
        self.fc_1 = nn.Dense(features=500)
        self.fc_2 = nn.Dense(features=self.latent_dims)

    def __call__(self, x):
        x = nn.leaky_relu(self.conv_1(x))
        x = nn.leaky_relu(self.conv_2(x))
        x = x.reshape((x.shape[0], -1))  # flatten
        x = nn.leaky_relu(self.fc_1(x))
        return self.fc_2(x)



In [59]:
h = AnalysisTransform(latent_dims=7*7*50)

In [62]:
h.__call__(test_image)

AttributeError: "AnalysisTransform" object has no attribute "conv_1". If "conv_1" is defined in '.setup()', remember these fields are only accessible from inside 'init' or 'apply'.

In [50]:
# Initialize the model parameters
rng = jax.random.PRNGKey(0)
model = MnistCompressionTrainer(latent_dims=10)
params = model.init(rng, test_image)

# Apply the analysis transform
z = model.apply({'params': params}, test_image)


TypeError: 'ContinuousEntropyModel' object is not callable

In [ ]:
codex.ops.soft_round_conditional_mean

In [5]:
class MnistCompressionTrainer(nn.Module):
    latent_dims: int

    def setup(self):
        self.analysis_transform = AnalysisTransform(self.latent_dims)
        self.synthesis_transform = SynthesisTransform()
        self.entropy_model = codex.ems.ContinuousEntropyModel()
    
    def __call__(self, x):
        z = self.analysis_transform(x)
        x_tilde = self.synthesis_transform(z)
        likelihood = self.entropy_model(z)
        return x_tilde, likelihood

    def train_step(self, state, batch):
        def loss_fn(params):
            x, _ = batch
            x_tilde, likelihood = self.apply({'params': params}, x)
            mse = jnp.mean((x - x_tilde)**2)
            nll = -jnp.mean(likelihood)
            return mse + nll

        grad = jax.grad(loss_fn)(state.params)
        return state.apply_gradients(grads=grad)

def create_train_state(rng, learning_rate, latent_dims):
    model = MnistCompressionTrainer(latent_dims)
    params = model.init(rng, jnp.ones((1, 28, 28, 1)))
    tx = optax.adam(learning_rate)
    return train_state.TrainState.create(
        apply_fn=model.apply, params=params, tx=tx)

In [7]:
# Instantiate the trainer
rng = random.PRNGKey(0)
state = create_train_state(rng, learning_rate=1e-3, latent_dims=10)

# Load the dataset
ds = load_dataset("mnist")
ds = ds.with_format("jax")

# Training loop
for epoch in range(10):  # Replace with the number of epochs you want to train for
    start_time = time.time()
    for batch in ds["train"].iter(batch_size=32):
        state = state.train_step(batch)
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch} finished in {epoch_time} seconds.")

TypeError: 'ContinuousEntropyModel' object is not callable

In [ ]:
codex.ems.ContinuousEntropyModel